In [ ]:
import os
import json
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date, timedelta
from typing import List
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from dotenv import load_dotenv

load_dotenv()

def generate_linked_dashboard(tickers: List[str], days: int) -> str:
    """
    Generates a single interactive HTML dashboard with linked subplots for
    comparing multiple stocks. Clicking the legend toggles visibility on both charts.
    """
    try:
        # Step 1: Fetch and process data for all tickers
        all_data = {}
        for ticker in tickers:
            today = date.today()
            start_date = today - timedelta(days=days)
            data = yf.download(ticker, start=start_date, end=today, progress=False)
            if data.empty: continue
            
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)
            
            data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
            data.dropna(subset=['Close'], inplace=True)
            
            if len(data) >= 2:
                data['Cumulative Return'] = (data['Close'] / data['Close'].iloc[0]) - 1
                data['Daily Return'] = data['Close'].pct_change().dropna()
                all_data[ticker] = data
        
        if not all_data:
            return "FAIL: No valid data could be fetched for the provided tickers."

        # Step 2: Create a single figure with two subplots
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=("Comparative Cumulative Return", "Return Distribution")
        )

        for ticker in all_data.keys():
            df = all_data[ticker]
            fig.add_trace(go.Scatter(
                x=df.index, y=df['Cumulative Return'],
                name=ticker, legendgroup=ticker,
                mode='lines', line=dict(width=2)
            ), row=1, col=1)
            fig.add_trace(go.Box(
                y=df['Daily Return'], name=ticker,
                legendgroup=ticker, showlegend=False
            ), row=1, col=2)

        # Step 3: Style the figure
        fig.update_layout(
            template='plotly_dark',
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='#161b22',
            margin=dict(l=40, r=20, t=80, b=40),
            title=dict(
                text=f'Comparative Stock Analysis: {", ".join(tickers)}',
                y=0.98, x=0.5, xanchor='center', yanchor='top',
                font=dict(size=20, color='#58a6ff', family='system-ui, sans-serif')
            ),
            # --- LEGEND POSITION FIX ---
            # This moves the legend to the top-left inside the plot area
            legend=dict(
                yanchor="top",
                y=0.98,
                xanchor="left",
                x=0.02
            )
        )
        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#30363d')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#30363d')
        fig.update_yaxes(tickformat='.0%', row=1, col=1)
        
        dashboard_html = fig.to_html(full_html=False, include_plotlyjs='cdn', config={'displayModeBar': False})
        
        # Step 4: Assemble final HTML page
        dashboard_filename = "final_dashboard.html"
        html_template = f"""
        <!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Final Stock Dashboard</title>
        <style>
            body{{font-family:system-ui,sans-serif;background-color:#0d1117;color:#c9d1d9;margin:0;padding:20px}}
            .container{{background-color:#161b22;border:1px solid #30363d;border-radius:8px;padding:20px;box-sizing:border-box;}}
        </style>
        </head><body><div class="container">{dashboard_html}</div></body></html>
        """
        with open(dashboard_filename, 'w') as f:
            f.write(html_template)
        
        return f"SUCCESS: Final interactive dashboard for {list(all_data.keys())} created at '{dashboard_filename}'."

    except Exception as e:
        import traceback
        traceback.print_exc()
        return f"FAIL: An unexpected error occurred: {e}"

# --- Agent Setup ---
llm = Gemini(model="models/gemini-2.5-flash-lite")
Settings.llm = llm
tools = [FunctionTool.from_defaults(fn=generate_linked_dashboard)]
agent = ReActAgent.from_tools(tools, llm=llm, verbose=False)

# --- Prompt and Execution ---
prompt = "Show me a 3-month comparison of AMD and NVDA."

print(f"--- Running Final Agent ---")
response = agent.chat(prompt)

print("\n--- Agent Final Response ---")
print(response)